In [56]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [57]:
def get_organ_df(df):
    new_df = df.groupby("psid")[['psid','fold']].first().reset_index(drop=True)

    for organ in ['liver', 'spleen','kidney']:
        tmp = df[df['organ']==organ]
        _psid = tmp['psid'].values
        _healthy = tmp['healthy'].values
        _low = tmp['low'].values
        _high = tmp['high'].values
        _df = pd.DataFrame({'psid':_psid, f'{organ}_healthy':_healthy, f'{organ}_low':_low, f'{organ}_high':_high})
        new_df = new_df.merge(_df, how='left', on='psid')
    new_df['patient_id'] = new_df['psid'].apply(lambda x: int(x.split("_")[0]))
    return new_df

def get_bowel_df(df):
    df = df[['psid','fold','preds']].rename(columns={'preds':'bowel_injury'})
    df['bowel_healthy'] = 1 - df['bowel_injury']
    df['patient_id'] = df['psid'].apply(lambda x: int(x.split('_')[0]))
    return df

def get_extra_df(df):
    df = df.rename(columns={'extra_output':'extravasation_injury'})
    df['extravasation_healthy'] = 1 - df['extravasation_injury']
    df['patient_id'] = df['psid'].apply(lambda x: int(x.split('_')[0]))
    return df

def get_solution_df(df):
    df['psid'] = df['patient_id'].astype(str) + '_' + df['series_id'].astype(str)
    for organ in ['liver','spleen','kidney']:
        df[f'{organ}_weight'] = np.where(df[f"{organ}_healthy"] == 1, 1, 
            np.where(df[f'{organ}_low'] == 1, 2, 4))
    df[f'bowel_weight'] = np.where(df[f"bowel_healthy"] == 1, 1, 2)
    df[f'extravasation_weight'] = np.where(df[f"extravasation_healthy"] == 1, 1, 6)
    df[f'any_injury_weight'] = np.where(df[f"any_injury"] == 1, 6, 1)
    df = df.drop(columns=['aortic_hu','incomplete_organ','png_suffix','dcm_folder',])
    return df

def get_submission_df(organ_df, bowel_df, extra_df):
    df = organ_df.merge(bowel_df[['psid','bowel_injury','bowel_healthy']], how='left', on='psid')
    df = df.merge(extra_df[['psid','extravasation_injury','extravasation_healthy']], how='left', on='psid')
    return df

In [58]:
# default 
if False:
    solution_path = "../../data/train_df.csv"

    organ_valid_path = "../../results/stage2-organ-type1/organ/organ_valid.csv"
    bowel_valid_path = "../../results/stage2-bowel-type1/bowel/bowel_valid.csv"
    extra_valid_path = "../../results/stage2-extra-type1-seg/extra-feat-sliding-seg-seq/extra_valid.csv"

    _solution = pd.read_csv(solution_path)

    organ_valid = pd.read_csv(organ_valid_path)
    bowel_valid = pd.read_csv(bowel_valid_path)
    extra_valid = pd.read_csv(extra_valid_path)

    organ_df = get_organ_df(organ_valid)
    bowel_df = get_bowel_df(bowel_valid)
    extra_df = get_extra_df(extra_valid)

    submission = get_submission_df(organ_df, bowel_df, extra_df)
    solution = get_solution_df(_solution)

    submission.to_csv("submission.csv", index=False)
    solution.to_csv("solution.csv", index=False)

In [70]:
# Ensemble & check Kaggle submissions
if True:
    solution_path = "../../data/train_df.csv"

    effv2s_organ_path = "../../results/stage2-organ-type1/effv2s/organ_valid.csv"
    effv2s_bowel_path = "../../results/stage2-bowel-type1/effv2s/bowel_valid.csv"
    seres_organ_path = "../../results/stage2-organ-type1/seresnext50_32x4d/organ_valid.csv"
    seres_bowel_path = "../../results/stage2-bowel-type1/seresnext50_32x4d/bowel_valid.csv"
    extra_valid_path = "../../results/stage2-extra-type1-seg/extra-feat-sliding-seg-seq/extra_valid.csv"

    # load valid df
    _solution = pd.read_csv(solution_path)

    effv2s_organ = pd.read_csv(effv2s_organ_path)
    effv2s_bowel = pd.read_csv(effv2s_bowel_path)
    seres_organ = pd.read_csv(seres_organ_path)
    seres_bowel = pd.read_csv(seres_bowel_path)
    extra_valid = pd.read_csv(extra_valid_path)

    effv2s_organ = get_organ_df(effv2s_organ)
    effv2s_bowel = get_bowel_df(effv2s_bowel)
    seres_organ = get_organ_df(seres_organ)
    seres_bowel = get_bowel_df(seres_bowel)
    extra_df = get_extra_df(extra_valid)

    organ_df = effv2s_organ[['psid','fold']].copy()
    organ_df[effv2s_organ.columns[2:-1]] = effv2s_organ.iloc[:,2:-1] * 0.8 + seres_organ.iloc[:,2:-1] * 0.2
    organ_df['patient_id'] = effv2s_organ['patient_id']
    
    bowel_df = effv2s_bowel[['psid','fold']].copy()
    bowel_df[effv2s_bowel.columns[2:-1]] = effv2s_bowel.iloc[:,2:-1] * 0.8 + seres_bowel.iloc[:,2:-1] * 0.2
    bowel_df['patient_id'] = effv2s_bowel['patient_id']

    submission = get_submission_df(organ_df, bowel_df, extra_df)
    solution = get_solution_df(_solution)

    submission.to_csv("submission.csv", index=False)
    solution.to_csv("solution.csv", index=False)